In [1]:
import pandas as pd
import numpy as np
import time
from geotext import GeoText
import collections
import math
import time
import re


In [2]:
### load data
adr = pd.read_csv('data/addresses.csv')
cnt = pd.read_csv('data/countries.csv')

In [3]:
adr.drop_duplicates(subset=['address'],inplace=True, keep='last')

In [4]:
def findCountryCode(istr):
    if istr in cnts: return cnts[istr]
    return np.NaN  

def findCountry(istr):
    istr = istr.lower()
    istr = re.sub(r'[^a-z]+', r' ', istr).strip()
    words = istr.split(' ')
    for w in words:
        if w in cnts: return w
    return np.NaN    

def findCity(istr):
    words = istr.split(' ')
    for w in words:
        w = w.strip()
        if w.islower() or w == 'University':
            continue
        cities = GeoText(w).cities
        if len(cities):
            return cities[0]
    return np.NaN

def fillGap(istr):
    if istr is not np.NaN and istr in mixed_dict:
        return mixed_dict[istr]
    else: return np.NaN
    
def processDublicates(istr):
    istr = istr.lower()
    istr = re.sub(r'[^a-z]+', r' ', istr)
    final_string = (' '.join("" if i in city_country_dict else i for i in istr.split(' '))).strip()   
    return final_string

In [5]:
cnt.Name = cnt.Name.str.lower()
cnt.set_index('Name', inplace=True)
cnts = cnt['Code'].to_dict()

In [6]:
adr['country'] = adr['address'].map(findCountry)
adr['city'] = adr['address'].map(findCity)

In [7]:
### filter dicts
city_dict = adr[['city', 'country']].to_dict('index')
mixed_dict = {}
for i in city_dict.keys():
    if isinstance(city_dict[i]['city'], str) and isinstance(city_dict[i]['country'], str):
        mixed_dict[city_dict[i]['city']] = city_dict[i]['country']
city_country_dict = set(list(mixed_dict.keys()) + list(mixed_dict.values()))

In [8]:
adr.loc[adr['country'].isnull(), 'country'] = adr['city'].map(fillGap)

In [9]:
### filter dublicates
adr['tmp'] = adr.address.map(processDublicates)
adr.drop_duplicates(subset='tmp', keep='last', inplace=True)
adr.drop('tmp', axis=1, inplace=True)

In [10]:
### get country code
adr['country'] = adr['country'].map(findCountryCode)

In [13]:
adr.to_csv('result_df.csv')